# Example 2
## Step 0 - prepare your data

Prepare cellphoneDB inputs starting from a seurat object object

In [2]:
library(Seurat)
library(SeuratObject)
library(Matrix)

Registered S3 methods overwritten by 'SeuratObject':
  method                     from  
  $.JackStrawData            Seurat
  $.Seurat                   Seurat
  $.SeuratCommand            Seurat
  $<-.Seurat                 Seurat
  [.Assay                    Seurat
  [.DimReduc                 Seurat
  [.Seurat                   Seurat
  [.SeuratCommand            Seurat
  [.SpatialImage             Seurat
  [[.Assay                   Seurat
  [[.DimReduc                Seurat
  [[.Seurat                  Seurat
  levels<-.Seurat            Seurat
  .DollarNames.JackStrawData Seurat
  .DollarNames.Seurat        Seurat
  .DollarNames.SeuratCommand Seurat
  as.list.SeuratCommand      Seurat
  as.logical.JackStrawData   Seurat
  dim.Assay                  Seurat
  dim.DimReduc               Seurat
  dim.Neighbor               Seurat
  dim.Seurat                 Seurat
  dim.SpatialImage           Seurat
  dimnames.Assay             Seurat
  dimnames.DimReduc          Seurat
  dimnames.

### 1. Load seurat object

The seurat object contains counts that have been normalized (per cell) and log-transformed.
If your data are raw counts, please normalize accordingly

In [3]:
so = readRDS('endometrium_example_counts_seurat.rds')

In [4]:
so

An object of class Seurat 
20975 features across 1949 samples within 1 assay 
Active assay: RNA (20975 features, 0 variable features)

In [5]:
Idents(so) = so$cell_type

In [13]:
# # If your data is not normalize, do it accordinlgy
# # For example
# so <- NormalizeData(object = so)

### 2. Write gene expression in mtx format


In [6]:
# Save normalised counts - NOT scaled!
writeMM(so@assays$RNA@data, file = 'endometrium_example_counts_mtx/matrix.mtx')
# save gene and cell names
write(x = rownames(so@assays$RNA@data), file = "endometrium_example_counts_mtx/features.tsv")
write(x = colnames(so@assays$RNA@data), file = "endometrium_example_counts_mtx/barcodes.tsv")

NULL

### 3. Generate your meta

In this example, our input is an anndata containing the cluster/celltype information in `metadat$'cell_type'`

The object also has `metadat$'lineage'` information wich will be used below for a hierarchical DEGs approach. 

In [14]:
table(so@meta.data$cell_type)


      Endothelial ACKR1      Endothelial SEMA3G           Fibroblast C7 
                    100                     100                     100 
          Fibroblast dS           Fibroblast eS                Lymphoid 
                    100                     100                     100 
                Myeloid                PV MYH11               PV STEAP4 
                    100                     100                     100 
           epi_Ciliated       epi_Ciliated LRG5           epi_Glandular 
                    100                     100                     100 
epi_Glandular_secretory           epi_Lumenal 1           epi_Lumenal 2 
                    100                     100                     100 
       epi_Pre-ciliated                epi_SOX9           epi_SOX9_LGR5 
                     49                     100                     100 
        epi_SOX9_prolif                    uSMC 
                    100                     100 

In [8]:
so@meta.data$Cell = rownames(so@meta.data)
df = so@meta.data[, c('Cell', 'cell_type')]
write.table(df, file ='endometrium_example_meta.tsv', sep = '\t', quote = F, row.names = F)

### 4. Compute DEGs (optional)

Use Seurat `FindAllMarkers` to compute differentially expressed genes and extract the corresponding data frame `DEGs`.
Here there are three options you may be interested on:
1. Identify DEGs for each cell type (compare cell type vs rest, most likely option) 
2. Identify DEGs for each cell type using a per-lineage hierarchycal approach (compare cell type vs rest in the lineage, such as in endometrium paper Garcia-Alonso et al 2021)

In the endometrium paper (Garcia-Alonso et al 2021) we're interested in the differences within the stromal and epithelial lineages, rather than the commonalities (example, what is specific of epithelials in the glands compared to epithelials in the lumen). The reason is that epithelial and stromal subtypes vary in space and type and thus we wanna extract the subtile differences within the lineage to better understand their differential location/ biological role.


In [16]:
## OPTION 1 - compute DEGs for all cell types
## Extract DEGs for each cell_type
# DEGs <- FindAllMarkers(so, 
#                        test.use = 'LR', 
#                        verbose = F, 
#                        only.pos = T, 
#                        random.seed = 1, 
#                        logfc.threshold = 0.2, 
#                        min.pct = 0.1, 
#                        return.thresh = 0.05)


# OPTION 2 - optional - Re-compute  hierarchical (per lineage) DEGs for Epithelial and Stromal lineages
DEGs = c()
for( lin in c('Epithelial', 'Stromal') ){
    message('Computing DEGs within linage ', lin)
    so_in_lineage = subset(so, cells = Cells(so)[ so$lineage == lin ] )
    celltye_in_lineage = unique(so$cell_type[ so$lineage == lin ])
    DEGs_lin = FindAllMarkers(so_in_lineage,
                       verbose = F, 
                       only.pos = T, 
                       random.seed = 1, 
                       logfc.threshold = 0, 
                       min.pct = 0.1, 
                       return.thresh = 0.05)
    DEGs = rbind(DEGs_lin, DEGs)
}

Computing DEGs within linage Epithelial

For a more efficient implementation of the Wilcoxon Rank Sum Test,
(default method for FindMarkers) please install the limma package
--------------------------------------------
install.packages('BiocManager')
BiocManager::install('limma')
--------------------------------------------
After installation of limma, Seurat will automatically use the more 
efficient implementation (no further action necessary).
This message will be shown once per session



In [18]:
'DKK1' %in% rownames(so@assays$RNA@counts)

[1] TRUE

In [10]:
fDEGs = subset(DEGs, p_val_adj < 0.05 & avg_logFC > 0.1)

# 1st column = cluster; 2nd column = gene 
fDEGs = fDEGs[, c('cluster', 'gene', 'p_val_adj', 'p_val', 'avg_logFC', 'pct.1', 'pct.2')] 
write.table(fDEGs, file ='endometrium_example_DEGs.tsv', sep = '\t', quote = F, row.names = F)

In [12]:
head(fDEGs)

,cluster,gene,p_val_adj,p_val,avg_logFC,pct.1,pct.2
,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSN,Fibroblast C7,GSN,4.870313e-35,2.321961e-39,1.9457498,0.91,0.725
IGFBP5,Fibroblast C7,IGFBP5,8.424704e-32,4.016545e-36,2.4430177,0.94,0.520
RPL21,Fibroblast C7,RPL21,5.587104e-31,2.663697e-35,0.9103342,1.00,0.990
RPS27,Fibroblast C7,RPS27,1.667606e-29,7.950448e-34,0.6828038,1.00,1.000
ASPN,Fibroblast C7,ASPN,6.253001e-26,2.981168e-30,1.8471276,0.55,0.025
C7,Fibroblast C7,C7,2.526954e-24,1.204746e-28,1.9273415,0.51,0.020


### 4. Run cellphoneDB

```
cellphonedb method degs_analysis  \
    endometrium_example_meta.tsv  \
    endometrium_example_counts_mtx  \
    endometrium_example_DEGs.tsv  \
    --microenvs endometrium_example_microenviroments.tsv  \ #optional
    --counts-data hgnc_symbol  \
    --database database/database/cellphonedb_user_2021-06-29-11_41.db \
    --threshold 0.1 
```

In [11]:
sessionInfo()

R version 4.0.3 (2020-10-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.1 LTS

Matrix products: default
BLAS:   /home/jovyan/my-conda-envs/R_Seurat3/lib/libblas.so.3.8.0
LAPACK: /home/jovyan/my-conda-envs/R_Seurat3/lib/liblapack.so.3.8.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] Matrix_1.3-2       SeuratObject_4.0.0 Seurat_3.2.2      

loaded via a namespace (and not attached):
  [1] nlme_3.1-152         matrixStats_0.58.0   RcppAnnoy_0.0.18    
  [4] RColorBrewer_1.1-2   httr_1.4.2           repr_1.1.3          
  [7